In [3]:
from gc import callbacks
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import History
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib
matplotlib.use('WebAgg')
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os

# Load your dataset
gold_price_data = pd.read_csv('data/harga_emas_new2.csv')
gold_price_data['Tanggal'] = pd.to_datetime(gold_price_data['Tanggal'])
gold_price_data.set_index('Tanggal', inplace=True)

gold_price_data['Harga'] = gold_price_data['Price1'].astype(str).str.replace('.', '').astype(float)
gold_price_data.drop(['Price1', 'Price2', 'Price3', 'Price5', 'Price10', 'Price25', 'Price50', 'Price100'], axis=1, inplace=True)

# Preprocess your dataset
def preprocess_data(df, look_back):
    data = df['Harga'].values
    data = data.reshape(-1, 1)
    
    # Normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data)

    train_size = int(len(data) * 0.8)
    train, test = data[0:train_size], data[train_size:len(data)]

    def create_dataset(dataset, look_back=1):
        X, Y = [], []
        for i in range(len(dataset) - look_back - 1):
            a = dataset[i:(i + look_back), 0]
            X.append(a)
            Y.append(dataset[i + look_back, 0])
        return np.array(X), np.array(Y)

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    # Reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], look_back, 1))
    testX = np.reshape(testX, (testX.shape[0], look_back, 1))

    return trainX, trainY, testX, testY, scaler

look_back = 10  # Adjust based on your requirements
trainX, trainY, testX, testY, scaler = preprocess_data(gold_price_data, look_back)

# Define the function to create the model
def create_model(lstm_units=150, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(lstm_units, return_sequences=True, input_shape=(look_back, 1)))
    model.add(LSTM(lstm_units // 2, return_sequences=False))
    model.add(Dense(25))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Define the parameter grid
param_grid = {
    'model__lstm_units': [100, 150, 200],
    'model__learning_rate': [0.001, 0.01],
    'model__dropout_rate': [0.2, 0.3],
    'batch_size': [20, 30, 50],
    'epochs': [30]
}

# model_path = 'saved_test_model/model.h5'

# Create the KerasRegressor
model = KerasRegressor(model=create_model, verbose=0)

# Create the GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)

# Fit the GridSearchCV
history = History()
grid_result = grid.fit(trainX, trainY, callbacks=[history])

# grid_result.save(model_path)

# Print the best parameters
print(f"Best Parameters: {grid_result.best_params_}")
print(f"Best Score: {grid_result.best_score_}")

Fitting 3 folds for each of 36 candidates, totalling 108 fits


c:\Users\LENOVO\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Parameters: {'batch_size': 20, 'epochs': 30, 'model__dropout_rate': 0.2, 'model__learning_rate': 0.001, 'model__lstm_units': 100}
Best Score: 0.9607191895856039


In [4]:
# Plot the training and validation loss of the best model
best_model = grid_result.best_estimator_.model
history = best_model.history

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

AttributeError: 'function' object has no attribute 'history'